In [7]:
import numpy as np
import pandas as pd
from datetime import datetime
import time

# scraping
import requests
from lxml import html
#from bs4 import BeautifulSoup

# visualization
import matplotlib.pyplot as plt

#supress format
#pd.options.display.float_format = '{:,.3f}'.format

In [8]:
# symbols
#symbols = ['AAPL', 'AMZN']#, 'NFLX', 'BABA', 'MSFT']
symbols = ['BAC']

yf_url = 'https://sg.finance.yahoo.com/quote/{}/{}?p={}'

In [17]:
def get_page(url):
    # headers to simulate browser
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    }
    return requests.get(url, headers=headers)

def parse_rows(table_rows):  
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)

    return pd.DataFrame(parsed_rows)

def clean_data(df):
    
    new_header = df.iloc[0] 
    df = df[1:] 
    df.columns = new_header
    
    return df

def scrape_table(url):  
    page = get_page(url)
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
    
    return df

def scrape_html_tables(url):
    page = get_page(url)
    stats_list = pd.read_html(page.content)
    
    return stats_list

## Retrieving Functions ##

def get_and_clean_table(df):
    
    df = df.set_index('Breakdown')
    df.replace(',', '', regex=True, inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.drop('ttm', axis=1, inplace=True)
    
    # reverses order of columns
    df = df.iloc[:, ::-1]
    
    return df

def get_pnl(symbol):
    
    df = scrape_table(yf_url.format(symbol, 'financials', symbol))
    df = get_and_clean_table(df)
    
    return df

def get_balance_sheet(symbol):
    
    df = scrape_table(yf_url.format(symbol, 'balance-sheet', symbol))
    df = get_and_clean_table(df)
    
    return df

def get_cash_flow(symbol):
    
    df = scrape_table(yf_url.format(symbol, 'cash-flow', symbol))
    df = get_and_clean_table(df)
    
    return df

def get_key_stats(symbols):
    
    key_stats_df = pd.DataFrame()

    for symbol in symbols:
        key_stats = pd.DataFrame()

        tables = scrape_html_tables(yf_url.format(symbol, 'key-statistics', symbol))

        for table in tables:
            key_stats = key_stats.append(table, ignore_index=True)

        key_stats.rename(columns={0: 'Stat', 1: symbol}, inplace=True)
        key_stats.set_index('Stat', inplace=True)
        
        key_stats_df = pd.concat([key_stats_df, key_stats], axis=1)
    
    return key_stats_df

def get_analysis_reports(symbol):
    
    analysis_reports = scrape_html_tables(yf_url.format(symbol, 'analysis', symbol))
    
    earnings_estimate = analysis_reports[0]
    revenue_estimate = analysis_reports[1]
    earnings_history = analysis_reports[2]
    eps_trend = analysis_reports[3]
    eps_revisions = analysis_reports[4]
    growth_estimates = analysis_reports[5]
    
    #for report in analysis_reports:
        #display(report)
    
    return analysis_reports
    #return earnings_estimate, revenue_estimate, earnings_history, eps_trend, eps_revisions, growth_estimates

def get_fcf(symbol):
    
    df = get_cash_flow(symbol).loc[['Net income', 'Operating cash flow', 'Capital expenditure']]
    df.loc['FCF to Equity (simple)'] = df.loc['Operating cash flow'] + df.loc['Capital expenditure']
    df.loc['FCFE/Net Income (%)'] = df.loc['FCF to Equity (simple)'] / df.loc['Net income'] * 100
    
    return df
    
def get_eps(symbols):
    
    key_stats_df = get_key_stats(symbols)
    eps = key_stats_df.loc[['Diluted EPS (ttm)']]
    
    return eps
    
def get_growth_rate(symbol):
    
    analysis_reports = get_analysis_reports(symbol)
    growth_estimates = analysis_reports[5].set_index('Growth estimates')
    
    growth_rate = float(growth_estimates.loc['Next 5 years (per annum)'][0].strip('%'))
    
    return growth_rate

def get_market_cap(symbols):
    
    key_stats_df = get_key_stats(symbols)
    market_cap = key_stats_df.loc[['Market cap (intra-day) 5']]
    
    #change to  numbers
    
    return market_cap

def get_estimated_revenue(symbol):
    
    analysis_reports = get_analysis_reports(symbol)
    estimated_revenue = analysis_reports[1].set_index('Revenue estimate').loc[['Avg. Estimate']]
    estimated_revenue.drop(estimated_revenue.columns[:2], axis=1, inplace=True)
    estimated_revenue.replace('B', '', regex=True, inplace=True)
    #estimated_revenue.replace('M', '', regex=True, inplace=True)
    
    #
    estimated_revenue = estimated_revenue.apply(pd.to_numeric, errors='coerce') * 1000000
    
    return estimated_revenue

def get_revenue(symbol):
    
    total_revenue = get_pnl(symbol).loc[['Total revenue']].reset_index(drop=True)
    estimated_revenue = get_estimated_revenue(symbol).reset_index(drop=True)
    
    df = pd.concat([total_revenue, estimated_revenue], axis=1)
    df.rename(index={0: 'Total Revenue'}, inplace=True)
    
    return df

def get_projected_revenue(symbol):
    
    revenue = get_revenue(symbol)
    revenue_growth_rate = revenue.T.pct_change() * 100
    revenue_growth_rate.rename(columns={'Total Revenue': 'Revenue Growth Rate'}, inplace=True)
    avg_revenue_growth_rate = revenue_growth_rate.mean()[0]
    
    #projecting next 2 years rev based on avg growth of past years
    revenue['2022E'] = revenue.iloc[:, -1] * (1 + avg_revenue_growth_rate/100)
    revenue['2023E'] = revenue.iloc[:, -1] * (1 + avg_revenue_growth_rate/100)
    
    return revenue

def get_net_income(symbol):
    
    past_revenue = get_pnl(symbol).loc[['Total revenue']]
    net_income = get_cash_flow(symbol).loc[['Net income']]
    df = pd.concat([past_revenue, net_income])
    df.loc['Net income margin'] = df.loc['Net income'] / df.loc['Total revenue'] * 100

    return df

# Discounted Cash Flow (DCF)

# Free Cash Flow

In [18]:
fcf = get_fcf('PAYS')
fcf

,31/12/2016,31/12/2017,31/12/2018,31/12/2019
Breakdown,,,,
Net income,1401.000000,1791.00000,2588.000000,7454.000000
Operating cash flow,1267.000000,7152.00000,15996.000000,16713.000000
Capital expenditure,-887.009000,-1519.00000,-1595.000000,-3237.000000
FCF to Equity (simple),379.991000,5633.00000,14401.000000,13476.000000
FCFE/Net Income (%),27.122841,314.51703,556.452859,180.788838


## video adds net borrowings, which needs to be scraped from seekingalpha.com

In [19]:
min_fcfe_ni_percent = min(fcf.loc['FCFE/Net Income (%)'])
min_fcfe_ni_percent

27.12284082798002

# Projecting Revenue based on Avg. Rev. Growth Rate

In [20]:
projected_revenue = get_projected_revenue('PAYS')
projected_revenue

,31/12/2016,31/12/2017,31/12/2018,31/12/2019,Current year (2020),Next year (2021),2022E,2023E
Total Revenue,10417.0,15234.0,23424.0,34667.0,45770.0,64490.0,92985.364801,134071.609043


# Calculating Net Income Margin

In [23]:
net_income = get_net_income('PAYS')
net_income

,31/12/2016,31/12/2017,31/12/2018,31/12/2019
Breakdown,,,,
Total revenue,10417.00000,15234.000000,23424.000000,34667.000000
Net income,1401.00000,1791.000000,2588.000000,7454.000000
Net income margin,13.44917,11.756597,11.048497,21.501716


In [25]:
net_income_margin = net_income.loc['Net income margin'].mean()
net_income_margin

14.438995077362227

# Projected Net Income

In [27]:
projected_revenues = projected_revenue.iloc[:, -4:]
projected_revenues.loc['Net income'] = projected_revenues.loc['Total Revenue'] * net_income_margin/100
projected_revenues

,Current year (2020),Next year (2021),2022E,2023E
Total Revenue,45770.000000,64490.000000,92985.364801,134071.609043
Net income,6608.728047,9311.707925,13426.152246,19358.593030


In [28]:
projected_revenues.loc['Free Cash Flow'] = projected_revenues.loc['Net income'] * low_fcfe_ni_percent/100
projected_fcf = projected_revenues
projected_fcf

,Current year (2020),Next year (2021),2022E,2023E
Total Revenue,45770.000000,64490.000000,92985.364801,134071.609043
Net income,6608.728047,9311.707925,13426.152246,19358.593030
Free Cash Flow,1792.474789,2525.599719,3641.553903,5250.600374


# Merge with past data

In [29]:
past_fcf = fcf.loc[['Net income', 'FCF to Equity (simple)']]
past_fcf.rename(index={'FCF to Equity (simple)': 'Free Cash Flow'}, inplace=True)
past_fcf

,31/12/2016,31/12/2017,31/12/2018,31/12/2019
Breakdown,,,,
Net income,1401.000,1791.0,2588.0,7454.0
Free Cash Flow,379.991,5633.0,14401.0,13476.0


In [32]:
projected_revenue.rename(index={'Total revenue': 'Total Revenue'}, inplace=True)
past_fcf = pd.concat([projected_revenue, past_fcf])
past_fcf

,31/12/2016,31/12/2017,31/12/2018,31/12/2019,Current year (2020),Next year (2021),2022E,2023E
Total Revenue,10417.000,15234.0,23424.0,34667.0,45770.0,64490.0,92985.364801,134071.609043
Net income,1401.000,1791.0,2588.0,7454.0,NaN,NaN,NaN,NaN
Free Cash Flow,379.991,5633.0,14401.0,13476.0,NaN,NaN,NaN,NaN


In [40]:
combined_fcf = pd.concat([past_fcf, projected_fcf], axis=1)
combined_fcf

,30/09/2016,30/09/2017,30/09/2018,30/09/2019,Current year (2020),Next year (2021),2022E,2023E
Breakdown,,,,,,,,
Total Revenue,"215,639,000.000","229,234,000.000","265,595,000.000","260,174,000.000","263,690,000.000","295,660,000.000","315,528,876.300","336,732,976.322"
Net income,"45,687,000.000","48,351,000.000","59,531,000.000","55,256,000.000","56,648,203.458","63,516,279.853","67,784,686.494","72,339,937.631"
Free Cash Flow,"52,276,000.000","50,803,000.000","64,121,000.000","58,896,000.000","59,520,975.374","66,737,349.080","71,222,217.285","76,008,476.587"


# Required Rate of Return (WACC)

## WACC = w_d * r_d * (1-t) + w_e * r_e

## cost of debt (r_d)

In [41]:
#interest expense from the past year (2019)

interest_expense = pnl.set_index('Breakdown').loc[['Interest expense']].iloc[:, 1][0]
interest_expense = pd.to_numeric(interest_expense.replace(',', ''))
interest_expense

3576000

In [42]:
pnl

,Breakdown,ttm,30/09/2019,30/09/2018,30/09/2017,30/09/2016
1,Total revenue,"267,981,000","260,174,000","265,595,000","229,234,000","215,639,000"
2,Cost of revenue,"165,854,000","161,782,000","163,756,000","141,048,000","131,376,000"
3,Gross profit,"102,127,000","98,392,000","101,839,000","88,186,000","84,263,000"
4,Research development,"17,383,000","16,217,000","14,236,000","11,581,000","10,045,000"
5,Selling general and administrative,"19,153,000","18,245,000","16,705,000","15,261,000","14,194,000"
6,Total operating expenses,"36,536,000","34,462,000","30,941,000","26,842,000","24,239,000"
7,Operating income or loss,"65,591,000","63,930,000","70,898,000","61,344,000","60,024,000"
8,Interest expense,"3,218,000","3,576,000","3,240,000","2,323,000","1,456,000"
9,Total other income/expenses net,"328,000","422,000","-441,000","-133,000","-1,195,000"
10,Income before tax,"67,091,000","65,737,000","72,903,000","64,089,000","61,372,000"


In [43]:
total_debt = balance_sheet.set_index('Breakdown').loc[['Current debt', 'Long-term debt']]#.iloc[:, 0]
total_debt.replace(',', '', regex=True, inplace=True)

total_debt = total_debt.apply(pd.to_numeric, errors='coerce')

total_debt.loc['Total debt'] = total_debt.loc['Current debt'] + total_debt.loc['Long-term debt']

total_debt

,30/09/2019,30/09/2018,30/09/2017,30/09/2016
Breakdown,,,,
Current debt,16240000,20748000,18473000,11605000
Long-term debt,91807000,93735000,97207000,75427000
Total debt,108047000,114483000,115680000,87032000


In [44]:
total_debt_ly = total_debt.iloc[2, 0]
total_debt_ly

108047000

In [45]:
r_d = interest_expense/total_debt_ly
r_d

0.033096707914148475

# Tax

## Taking income tax expense and divide over income before tax

In [46]:
income_tax = pnl.set_index('Breakdown').loc[['Income before tax', 'Income tax expense']]
income_tax.replace(',', '', regex=True, inplace=True)
income_tax = income_tax.apply(pd.to_numeric, errors='coerce')
income_tax.loc['Calc. Eff. Tax Rate'] = income_tax.loc['Income tax expense'] / income_tax.loc['Income before tax'] * 100
income_tax

,ttm,30/09/2019,30/09/2018,30/09/2017,30/09/2016
Breakdown,,,,,
Income before tax,"67,091,000.000","65,737,000.000","72,903,000.000","64,089,000.000","61,372,000.000"
Income tax expense,"9,876,000.000","10,481,000.000","13,372,000.000","15,738,000.000","15,685,000.000"
Calc. Eff. Tax Rate,14.720,15.944,18.342,24.556,25.557


In [47]:
income_tax_rate = income_tax.iloc[2, 1]
income_tax_rate

15.94383680423506

# CAPM

## R_a = R_f + b(R_m - R_f)

## Risk-free rate (R_f)

In [48]:
R_f = 0.6530/100
R_f

0.00653

## beta

In [49]:
stats_reports['AAPL']

,AAPL
Stat,
Market cap (intra-day) 5,1.66T
Enterprise value 3,1.68T
Trailing P/E,30.07
Forward P/E 1,25.77
PEG Ratio (5 yr expected) 1,2.70
Price/sales (ttm),6.19
Price/book (mrq),21.10
Enterprise value/revenue 3,6.28
Enterprise value/EBITDA 6,21.78


In [50]:
b = pd.to_numeric(stats_reports['AAPL'].loc['Beta (5Y monthly)'][0])
b

1.18

## R_m

## using avg. return of the s&p 500 - 10% (for most 10-year periods)

In [51]:
R_m = 0.1
R_a = R_f + b * (R_m - R_f)
R_a

0.1168246

## Weights for WACC formula

In [52]:
market_cap = stats_reports['AAPL'].loc['Market cap (intra-day) 5'][0]
#market_cap = market_cap.replace('T', '')
market_cap = pd.to_numeric(market_cap.replace('T', '')) * 1000000000

market_cap

1660000000.0

In [53]:
total_debt_ly = total_debt.iloc[2, 0]

In [54]:
w_d = total_debt_ly / (total_debt_ly + market_cap)
w_e = 1 - w_d
w_d

0.06111093200576682

In [55]:
'wacc = ' + str(round(w_d, 2)) + ' * ' + str(round(r_d, 2)) 

'wacc = 0.06 * 0.03'

In [56]:
r_e = R_a

wacc = w_d * r_d * (1 - income_tax_rate/100) + w_e * r_e
wacc

0.11138543511336552

## WACC is the required rate of return

## Oustanding shares

In [57]:
shares_outstanding = pd.to_numeric(stats_reports['AAPL'].loc['Shares outstanding 5'][0].replace('B', '')) * 1000000 
# rmb to reduce by 3 zeros as all other financial numbers are in thousands

shares_outstanding

4330000.0

## Perpetual growth rate - assume 2.5%

In [58]:
g = 0.025

In [59]:
fcf_2 = combined_fcf.loc[['Free Cash Flow']]
fcf_2['Terminal Value'] = ( fcf_2.iloc[:, -1][0] * (1 + g) ) / (wacc - g)
fcf_2

,30/09/2016,30/09/2017,30/09/2018,30/09/2019,Current year (2020),Next year (2021),2022E,2023E,Terminal Value
Breakdown,,,,,,,,,
Free Cash Flow,"52,276,000.000","50,803,000.000","64,121,000.000","58,896,000.000","59,520,975.374","66,737,349.080","71,222,217.285","76,008,476.587","901,872,965.036"


## Discount factor

In [60]:
fcf_3 = fcf_2.iloc[:, 4:]

#fcf_3.iloc['Discount Factor'] = 

fcf_3 = fcf_3.T
fcf_3['Discount Factor'] = (1 + wacc) ** (np.arange(len(fcf_3)) + 1)
fcf_3.iloc[4, 1] = fcf_3.iloc[3, 1] # change terminval value dc factor to time period 4

fcf_3

Breakdown,Free Cash Flow,Discount Factor
Current year (2020),"59,520,975.374",1.111
Next year (2021),"66,737,349.080",1.235
2022E,"71,222,217.285",1.373
2023E,"76,008,476.587",1.526
Terminal Value,"901,872,965.036",1.526


# PV of future cash flows

In [61]:
fcf_3['PV of Future CF'] = fcf_3['Free Cash Flow'] / fcf_3['Discount Factor']

fcf_3.T

,Current year (2020),Next year (2021),2022E,2023E,Terminal Value
Breakdown,,,,,
Free Cash Flow,"59,520,975.374","66,737,349.080","71,222,217.285","76,008,476.587","901,872,965.036"
Discount Factor,1.111,1.235,1.373,1.526,1.526
PV of Future CF,"53,555,655.395","54,030,570.074","51,882,558.809","49,819,942.763","591,134,851.205"


In [62]:
total_value_today = fcf_3['PV of Future CF'].sum()

value_today = total_value_today/shares_outstanding
value_today

184.85532984883764

# Move last column to first column

# Reverse order of columns

In [ ]:
# reverses order of columns
total_revenue = total_revenue.iloc[:, ::-1] 
total_revenue

# Multiple Reports for Multiple Stocks

# Get stats comparison btw multiple companies

# Alpha Vantage

# New Implementation (All symbols in 1 df)